### Tratamento do texto do campo DECISAO do dataset de Acórdãos

In [6]:
import re, os, signal

import servio_tools as st

In [2]:
# Configuração para mostrar todas as linhas de um dataframe
# pd.set_option('display.max_rows', None)

In [3]:
df_acordaos = st.read_file('dados/df_acordaos.csv')
df_acordaos.info()

Duration: 0:00:41.997628


In [11]:
# Aplicação do regex no texto do dataset de acordaos
start_time = datetime.now()
df_acordaos.DECISAO = df_acordaos.apply(lambda x: padronizacao(x), axis = 1)
print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:37:38.526097


In [12]:
st.save_file(df_acordaos, 'dados/df_acordaos_proc.csv'

Duration: 0:00:48.264869


In [ ]:
os.kill(os.getpid(), signal.SIGKILL)